In [1]:
import pandas as pd 
import pickle
import warnings
warnings.filterwarnings('ignore')
import numpy as np

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

import torch
import torch.nn as nn
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

In [2]:
labels = pd.read_csv("../../data/scene_labels.csv")

In [3]:
labels.head(5)

,SCENE,KEY,SHOW,Sarcasm,Sarcasm_Type
0,1_10004,1_10004_u,BBT,0.0,NONE
1,1_10009,1_10009_u,BBT,0.0,NONE
2,1_1001,1_1001_u,BBT,0.0,NONE
3,1_1003,1_1003_u,BBT,1.0,PRO
4,1_10190,1_10190_u,BBT,0.0,NONE


#### Perform mean, median, max, min and sum pooling on audio feature data

In [4]:
def get_model_data(audio_features):
    model_data = pd.DataFrame(columns=['audio_feature','sarcasm','sarcasm_type'])
    for index, row in labels.iterrows():
        audio_key = row["SCENE"] + "_u.wav"
        model_data = model_data.append({'audio_feature': audio_features[audio_key],
                                    'sarcasm' : row["Sarcasm"],
                                    'sarcasm_type' : row["Sarcasm_Type"]},
                                  ignore_index=True)
    return model_data

In [5]:
def get_train_test_split(model_data, x_column, y_column, stratify_column):
    train_count = 301
    train_data = model_data.groupby(stratify_column).apply(lambda x: x.sample(n=train_count, random_state=42))
    train_index_list = train_data.index.values.tolist()
    test_data = model_data[~model_data.index.isin(train_index_list)]
    train_data.reset_index(drop=True, inplace = True)
    test_data.reset_index(drop=True, inplace = True)
    return train_data, test_data

In [6]:
class GRUTensorDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        features = self.data.loc[index, 'padded_audio_feature']
        label = self.data.loc[index, 'sarcasm']
        return torch.from_numpy(features).float(), label
    
    def __getindexlist__(self):
        return list(self.data.index.values)
    
class GRUNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, 
                 output_dim, n_layers):
        super(GRUNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, 
                          n_layers, batch_first = True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x, h):
        out, h = self.gru(x, h)
        out = self.softmax(self.fc(out[:,-1]))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, 
                            self.hidden_dim).zero_()
        return hidden
    
def evaluateGRU(gru, review, size):
    hidden = gru.init_hidden(size)
    output, hidden = gru(review, hidden)
    return output

def categoryFromOutput(output):
    top_n, top_i = torch.max(output,dim=1)
    return top_i

def test_accuracy(gru, loader, size):
    actuals = []
    predictions = []
    for data, target in loader:
        output = evaluateGRU(gru, data, size)
        prediction_index = categoryFromOutput(output)
        predictions = prediction_index.tolist()
        actuals = target.tolist()
    return predictions, actuals
    
hidden_size = 18
output_size = 2
input_size = 690
n_layers = 1

### Librosa

In [7]:
with open('../../audio_features/feat_dict_librosa_lld.pickle', 'rb') as f:
    librosa_audio_features = pickle.load(f, encoding='latin1')
    
model_data = get_model_data(librosa_audio_features)

desired_length = 18

train_data, test_data = get_train_test_split(model_data, 'audio_feature', 'sarcasm', 'sarcasm')
fnn_train = train_data.copy()
fnn_test = test_data.copy()
fnn_train.reset_index(drop=True, inplace = True)
fnn_test.reset_index(drop=True, inplace = True)

        
fnn_train['padded_audio_feature'] = fnn_train.loc[:, 'audio_feature']
for index, row in fnn_train.iterrows():
    data_array = row['padded_audio_feature']
    new_array = []
    for arr in data_array:
        if arr.shape[0] < desired_length:
            arr = np.pad(arr, (0, desired_length - arr.shape[0]), 'constant')
            new_array.append(arr)
        else:
            new_array.append(arr)
    fnn_train.at[index, "padded_audio_feature"] = np.transpose(np.array(new_array))

fnn_test['padded_audio_feature'] = fnn_test.loc[:, 'audio_feature']
for index, row in fnn_test.iterrows():
    data_array = row['padded_audio_feature']
    new_array = []
    for arr in data_array:
        if arr.shape[0] < desired_length:
            arr = np.pad(arr, (0, desired_length - arr.shape[0]), 'constant')
            new_array.append(arr)
        else:
            new_array.append(arr)
    fnn_test.at[index, "padded_audio_feature"] = np.transpose(np.array(new_array))

fnn_train["sarcasm"] = fnn_train["sarcasm"].astype('int').to_numpy()
fnn_test["sarcasm"] = fnn_test["sarcasm"].astype('int').to_numpy()

In [8]:
fnn_train_tensor = GRUTensorDataset(fnn_train)
fnn_test_tensor = GRUTensorDataset(fnn_test)

num_of_workers = 0
batch_size = 43
valid_size = 0.1

train_indices = list(range(len(fnn_train_tensor)))
np.random.shuffle(train_indices)

test_indices = list(range(len(fnn_test_tensor)))
np.random.shuffle(test_indices)

train_loader = torch.utils.data.DataLoader(
    fnn_train_tensor, 
    batch_size=batch_size, 
    sampler=SubsetRandomSampler(train_indices)
)

test_loader = torch.utils.data.DataLoader(
    fnn_test_tensor, 
    batch_size=batch_size, 
    sampler=SubsetRandomSampler(test_indices)
)

In [9]:
gru = GRUNet(input_size, hidden_size, output_size, n_layers)
print(gru)
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(gru.parameters(), lr=0.001)

GRUNet(
  (gru): GRU(690, 18, batch_first=True)
  (fc): Linear(in_features=18, out_features=2, bias=True)
  (softmax): LogSoftmax(dim=1)
)


In [10]:
n_epochs = 1001
    
test_min_loss = np.inf

for epoch in range(n_epochs):
    torch.manual_seed(42)
    train_loss = 0.0
    test_loss = 0.0
    gru.train()
    for data, target in train_loader:
        h = gru.init_hidden(batch_size)
        optimizer.zero_grad()
        output, h = gru(data, h.data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)

    gru.eval()
    for data, target in test_loader:
        if data.shape[1] < 43:
            continue
        h = gru.init_hidden(batch_size)
        output, h = gru(data, h.data)
        loss = criterion(output, target)
        test_loss += loss.item()*data.size(0)

    train_loss = train_loss / len(train_loader.dataset)
    test_loss = test_loss / len(test_loader.dataset)
    
#     if(epoch%20 == 0):
# #         print(f"Epoch: {epoch+1:02}")
# #         print("\tTraining Loss: {:.6f} \Test Loss: {:.6f}".format(train_loss, test_loss))
#     if test_loss <= test_min_loss:
# #         print("Test loss decreased ({:.6f} --> {:.6f}). Saving model...".format(test_min_loss, test_loss))
# #         torch.save(gru.state_dict(), "fnnmodel.pt")
#         test_min_loss = test_loss
    if(epoch%50 == 0):
        torch.save(gru.state_dict(), "gru/gru_" + str(epoch) + "_.pt")
        print("Epoch: " + str(epoch))
        test_loader = torch.utils.data.DataLoader(fnn_test_tensor, batch_size=fnn_test_tensor.__len__())
        predictions, actuals = test_accuracy(gru, test_loader, fnn_test_tensor.__len__())
        print(pd.DataFrame(classification_report(actuals, predictions, output_dict=True)).T)
        test_loader = torch.utils.data.DataLoader(fnn_test_tensor, batch_size=batch_size, sampler=SubsetRandomSampler(test_indices))

Epoch: 0
              precision    recall  f1-score      support
0              0.000000  0.000000  0.000000   601.000000
1              0.499584  0.998336  0.665927   601.000000
accuracy       0.499168  0.499168  0.499168     0.499168
macro avg      0.249792  0.499168  0.332963  1202.000000
weighted avg   0.249792  0.499168  0.332963  1202.000000
Epoch: 50
              precision    recall  f1-score     support
0              0.696970  0.574043  0.629562   601.00000
1              0.637907  0.750416  0.689602   601.00000
accuracy       0.662230  0.662230  0.662230     0.66223
macro avg      0.667438  0.662230  0.659582  1202.00000
weighted avg   0.667438  0.662230  0.659582  1202.00000
Epoch: 100
              precision    recall  f1-score      support
0              0.707602  0.603993  0.651706   601.000000
1              0.654572  0.750416  0.699225   601.000000
accuracy       0.677205  0.677205  0.677205     0.677205
macro avg      0.681087  0.677205  0.675465  1202.000000
weighte